# Federated Learning with Differential Privacy

Please make sure you set up a virtual environment and follow [example root readme](../../README.md) before starting this notebook.
Then, install the requirements.

<div class="alert alert-block alert-info"> <b>NOTE</b> Some of the cells below generate long text output.  We're using <pre>%%capture --no-display --no-stderr cell_output</pre> to suppress this output.  Comment or delete this line in the cells below to restore full output.</div>

In [2]:
%%capture --no-display --no-stderr cell_output
!pip install -r requirements.txt

### Differential Privacy (DP)
[Differential Privacy (DP)](https://arxiv.org/abs/1910.00962) [7] is a method for ensuring that Federated Learning (FL) preserves privacy by obfuscating the model updates sent from clients to the central server. 
This example shows the usage of a CIFAR-10 training code with NVFlare, as well as the usage of DP filters in your FL training. DP is added as a filter in `config_fed_client.json`. Here, we use the "Sparse Vector Technique", i.e. the [SVTPrivacy](https://nvflare.readthedocs.io/en/main/apidocs/nvflare.app_common.filters.svt_privacy.html) protocol, as utilized in [Li et al. 2019](https://arxiv.org/abs/1910.00962) [7] (see [Lyu et al. 2016](https://arxiv.org/abs/1603.01699) [8] for more information).

## Run experiments with FL simulator
### Training with FL simulator
FL simulator is used to simulate FL experiments or debug codes, not for real FL deployment.

First, train a model using the FedAvg algorithm with four clients without DP.

In [ ]:
!nvflare simulator './configs/brats_fedavg' -w './workspace_brats/brats_fedavg' -n 4 -t 4

Run the FL simulator with 4 clients for federated learning with differential privacy by running

In [ ]:
!nvflare simulator './configs/brats_fedavg_dp' -w './workspace_brats/brats_fedavg_dp' -n 4 -t 4

### Testing with FL simulator
The best global models are stored at
```
workspace_brats/[job]/simulated_job/app_server/best_FL_global_model.pt
```

Please then add the correct paths to the testing script, and run

In [ ]:
!cd ./result_stat
!bash testing_models_3d.sh